In [1]:
import os
from dotenv import load_dotenv
# Loads environment variables from a .env file into your system's environment.
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(api_key=groq_api_key, model="Gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002326A735A90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002326A736C30>, model_name='Gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

#### Using manual update with AIMesage to see Conversation History

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Ravi, I am a Software Engineer and an aspiring AI Engineer")])

AIMessage(content="Hi Ravi, it's nice to meet you! \n\nThat's great that you're a software engineer and aspiring AI engineer. It's a really exciting field with lots of opportunities.  \n\nWhat aspects of AI are you most interested in? Do you have any specific projects or areas you're hoping to explore? \n\nI'm happy to chat about AI, answer any questions you have, or even help you brainstorm ideas if you'd like.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 26, 'total_tokens': 126, 'completion_time': 0.181818182, 'prompt_time': 0.002244774, 'queue_time': 0.230675411, 'total_time': 0.184062956}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0c3e74e1-ed0b-4ec5-ac56-37d06d4acfd7-0', usage_metadata={'input_tokens': 26, 'output_tokens': 100, 'total_tokens': 126})

In [4]:
from langchain_core.messages import AIMessage
# Using the `model.invoke` method to simulate a conversation with the chatbot.
# The conversation includes:
# 1. A HumanMessage introducing the user as Ravi, a Software Engineer and aspiring AI Engineer.
# 2. An AIMessage responding to Ravi's introduction.
# 3. A follow-up HumanMessage asking the chatbot to recall Ravi's profession.
# This demonstrates how the chatbot can remember and respond based on the conversation history.
model.invoke(
    [
        HumanMessage(content="Hi, My name is Ravi, I am a Software Engineer and an aspiring AI Engineer"),
        AIMessage(content="Hi Ravi, it's nice to meet you!\n\nIt's great to hear that you're a Software Engineer and aspiring AI Engineer."),
        HumanMessage(content="whats my profession?"),
    ]
) 
# So, this is used to remember conversation history and provide context-aware responses.

AIMessage(content="You told me your profession is a **Software Engineer** and that you are an aspiring **AI Engineer**. \n\nIs there anything else you'd like to know or discuss? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 70, 'total_tokens': 112, 'completion_time': 0.076363636, 'prompt_time': 0.003637161, 'queue_time': 0.235664751, 'total_time': 0.080000797}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bc198c20-08f6-4209-977f-a9195a74f20a-0', usage_metadata={'input_tokens': 70, 'output_tokens': 42, 'total_tokens': 112})

#### What is **Message History** in an AI Chatbot?

In the context of AI chatbots, **message history** refers to the **persistent or temporary memory of the past exchanges** between the user and the chatbot — including both the user's inputs and the model's outputs.

This **conversation history** helps make the chatbot **stateful**, meaning it can **"remember" previous messages** and **respond in a more coherent and context-aware way**.

---

#### Why is Message History Important?

Most LLMs (like GPT, Claude, etc.) are **stateless by default**, which means:
- Every prompt is treated **independently**, unless you **manually provide context**.
- Without message history, the bot would "forget" what was said earlier in the conversation.

So to **maintain context**, we pass prior messages along with the new user message in the prompt to the model.

#### Using ChatMessage History for Convertaion History

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# This is an in-memory store that maps each unique session ID (like a user's chat session) to its own ChatMessageHistory.
# Think of it like a cache that holds ongoing conversations by session_id.
store = {}

# This function checks if a message history exists for the given session, If not, it initializes a new ChatMessageHistory() object and stores it in store using the session_id as the key
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

#  Wrap your model with RunnableWithMessageHistory
# It calls your get_session_history() function to retrieve or initialize history for a given session ID, It loads all prior messages (Human and AI) from the history store
# It appends the new human message to that history, It sends the entire message list to the model or chain and It stores the model’s response as an AI message back into the history
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
# It creates a config dictionary that is passed to .invoke() in RunnableWithMessageHistory. This tells LangChain which conversation session to use
config = {"configurable":{"session_id":"chat1"}}

In [7]:
# If you use the same session_id, your chatbot will remember past messages — even across multiple .invoke() calls.
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Ravi, I am a Software Engineer and an aspiring AI Engineer"),
    ],
    config=config
)

In [8]:
print(response.content)

Hi Ravi, it's nice to meet you!

It's great to hear that you're a Software Engineer and aspiring AI Engineer.  That's a really exciting field to be in.  

Do you have any specific areas of AI that you're most interested in?  Or perhaps some projects you're working on or thinking about working on? I'd love to hear more about your goals and aspirations.



In [9]:
# Change the config with session id
# config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [
        HumanMessage(content="Whats my name")
    ],
    config=config
)

In [10]:
print(response.content)

Your name is Ravi.  

I remember that from our introduction! 😊 

Is there anything else I can help you with?



In [11]:
response = with_message_history.invoke(
    [
        HumanMessage(content="I wanted to be an IAS officer.")
    ],
    config=config
)

In [12]:
print(response.content)

That's fantastic, Ravi!  Becoming an IAS officer is a very admirable goal. 

It requires dedication, hard work, and a strong commitment to public service. What aspects of the role appeal to you the most?  

I'm happy to chat with you more about your aspirations if you'd like.  Perhaps you have questions about the process, or you want to brainstorm ways to prepare.  I'm here to listen and offer any support I can.



In [13]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Whats my profession and whats my future plan?")
    ],
    config=config
)

In [14]:
print(response.content)

Based on our conversation, you are currently a Software Engineer and aspire to become an AI Engineer. 

You also mentioned wanting to be an IAS officer, which shows you have diverse and ambitious goals! 

Is there one path you're currently focusing on more than the others? 


I'm here to help you explore any of these paths further if you'd like.



In [15]:
response = with_message_history.invoke(
    [
        HumanMessage(content="What I wanted to become?")
    ],
    config=config
)
print(response.content)

You said you wanted to become an IAS officer!  

It's a challenging but rewarding career path.  

Is there anything specific about that role that interests you?  Or perhaps you have some questions about what it takes to become an IAS officer? I'm happy to help you explore it further.



In [16]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Do you remember by Name?")
    ],
    config=config
)
print(response.content)

Yes, I remember your name is Ravi!  😊 

Is there anything else I can help you with today?



In [17]:
print(store)

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, My name is Ravi, I am a Software Engineer and an aspiring AI Engineer', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Ravi, it's nice to meet you!\n\nIt's great to hear that you're a Software Engineer and aspiring AI Engineer.  That's a really exciting field to be in.  \n\nDo you have any specific areas of AI that you're most interested in?  Or perhaps some projects you're working on or thinking about working on? I'd love to hear more about your goals and aspirations.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 26, 'total_tokens': 118, 'completion_time': 0.167272727, 'prompt_time': 0.002119915, 'queue_time': 0.23469589, 'total_time': 0.169392642}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-634c97e4-024d-4b1a-a25c-20c3aa82a55a-0', usage_metadata={'input_tokens